In [ ]:
# History_Bonds_Target_v4.3.1.py
# - Cleaned workbook (unchanged)
# - Target workbook (unchanged from 4.3)
# - Event Analysis:
#     * Sheet names = UST buckets (unique-suffixed)
#     * Axis dates PER BOND: min = first non-NaN date for that bond; max = last non-NaN date
#     * Chart 1: Prior solid line; Post square markers (same color), no connecting line
#     * Chart 2: Post-only; x-axis min = max(event_date, bond first non-NaN); max = last non-NaN
# - Output folder: Output {M-D-YYYY}
# - Colab auto-downloads: Cleaned/Target/Event at the end

import os, re, glob, shutil, math, subprocess
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta, time as dtime
from collections import defaultdict

# ---- Globals so the download block can see what was written ----
OUT_XLSX = None
TARGET_XLSX = None
EVENT_XLSX = None

# ---- Ensure xlsxwriter quietly ----
try:
    import xlsxwriter  # noqa: F401
except Exception:
    subprocess.run(["pip", "install", "-q", "xlsxwriter"], check=False)
    import xlsxwriter  # noqa: F401

# ---- Time/date + folders ----
try:
    from zoneinfo import ZoneInfo
    tz = ZoneInfo("America/New_York")
except Exception:
    tz = None

now = datetime.now(tz) if tz else datetime.now()
try:
    FS_DATE = now.strftime("%-m-%-d-%Y")
except ValueError:
    FS_DATE = now.strftime("%#m-%#d-%Y")

RAW_DIR    = f"Raw {FS_DATE}"
OUTPUT_DIR = f"Output {FS_DATE}"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# Common helpers
# =========================
COUPON_DECIMALS = 3
def _to_float_safe(x):
    try: return float(str(x).replace(",", "").strip())
    except Exception: return np.nan
def _format_coupon(c):
    if pd.isna(c): return ""
    val = round(float(c), COUPON_DECIMALS)
    return f"{val:.{COUPON_DECIMALS}f}".rstrip("0").rstrip(".")
LABEL_RE = re.compile(r"^\s*(?P<ticker>[A-Za-z0-9\.\-]+)\s+(?P<coupon>\d+(?:\.\d+)?)%\s+(?P<month>\d{1,2})[\/\-](?P<year>\d{2,4})\s*$")
def parse_bond_label(label: str | None):
    if not isinstance(label, str): return None
    s = re.sub(r"\s+(T[- ]?Spread|G[- ]?Spread|Confidence)\s*$", "", label.strip(), flags=re.IGNORECASE)
    s = re.sub(r"(\b\d{1,2})-(\d{2,4}\b)", r"\1/\2", s)
    m = LABEL_RE.match(s)
    if not m: return None
    d = m.groupdict()
    y = int(d["year"])
    year = y + (2000 if y < 80 else (0 if y >= 1900 else 1900))
    return {"ticker": d["ticker"].upper(), "coupon": _to_float_safe(d["coupon"]), "month": int(d["month"]), "year": year}
def canonical_label(d):
    if not d: return None
    return f'{d["ticker"]} {_format_coupon(d["coupon"])}% {d["month"]}/{d["year"]}'
def norm_ust(u):
    s = str(u).strip()
    if not s: return s
    if s.lower().startswith("perp"): return "Perp"
    m = re.match(r"(\d+)", s)
    return (m.group(1) + "y") if m else s

def months_between_inclusive(a: date, b: date) -> int:
    if a > b: a, b = b, a
    return (b.year - a.year) * 12 + (b.month - a.month) + 1

def configure_date_axis_monthly(chart, start_d: date, end_d: date):
    month_count = months_between_inclusive(start_d, end_d)
    major_unit = 2 if month_count >= 8 else 1
    chart.set_x_axis({
        'date_axis': True,
        'num_format': "mmm 'yy",
        'label_position': 'low',
        'major_unit': major_unit,
        'major_unit_type': 'months',
        'min': datetime.combine(start_d, dtime.min),
        'max': datetime.combine(end_d, dtime.min),
        'num_font': {'rotation': 0},
        'major_gridlines': {'visible': False},
        'minor_gridlines': {'visible': False},
    })

def y_range_5s_push10(vals) -> tuple[float, float]:
    arr = np.array(vals, dtype=float)
    arr = arr[~np.isnan(arr)]
    if arr.size == 0: return (0.0, 5.0)
    ymin, ymax = float(np.nanmin(arr)), float(np.nanmax(arr))
    down5 = lambda x: 5.0 * math.floor(x / 5.0)
    up5   = lambda x: 5.0 * math.ceil(x / 5.0)
    y_min, y_max = down5(ymin), up5(ymax)
    if y_min == y_max: y_max = y_min + 5.0
    if (y_max - y_min) % 10 != 0:
        y_max += (10 - ((y_max - y_min) % 10))
    return (y_min, y_max)

def base_chart(wb):
    ch = wb.add_chart({'type': 'line'})
    ch.set_title({'name': ''})
    ch.set_plotarea({'fill': {'none': True}, 'border': {'none': True}})
    ch.set_chartarea({'fill': {'none': True}, 'border': {'none': True}})
    ch.set_legend({'position': 'bottom'})
    return ch

# simple palette for Event Analysis
EA_COLORS = ["#1F497D", "#C0504D", "#9BBB59", "#8064A2", "#4BACC6", "#F79646", "#2F5597", "#A31515"]

# =========================
# File ingestion
# =========================
def get_raw_folder() -> str:
    resp = input("Do you want to upload new files? (y/n): ").strip().lower()
    if resp in {"y", "yes"}:
        if os.path.exists(RAW_DIR):
            shutil.rmtree(RAW_DIR)
        os.makedirs(RAW_DIR, exist_ok=True)
        try:
            from google.colab import files as colab_files
            print("Please upload your Excel files now...")
            uploaded = colab_files.upload()
            for fname, data in uploaded.items():
                with open(os.path.join(RAW_DIR, fname), "wb") as f:
                    f.write(data)
            print(f"✅ Saved {len(uploaded)} file(s) into: {RAW_DIR}")
        except Exception as e:
            raise SystemExit(f"Upload failed: {e}")
        return RAW_DIR
    else:
        raw_folders = sorted(glob.glob("Raw *"), key=os.path.getmtime)
        if not raw_folders:
            raise SystemExit("❗ No Raw folders found. Please upload new data.")
        selected = raw_folders[0]
        print(f"Using existing folder: {selected}")
        return selected

def ingest_bonds_and_history(raw_dir: str):
    paths = sorted(glob.glob(os.path.join(raw_dir, "*.xlsx")))
    if not paths:
        raise SystemExit("❗ No Excel files found in the selected Raw folder.")

    history_files, bonds_file = [], None
    for path in paths:
        try:
            xl = pd.ExcelFile(path)
        except Exception:
            continue
        if "Combined Level History" in xl.sheet_names:
            history_files.append((path, xl))
        else:
            try:
                df0 = pd.read_excel(xl, sheet_name=xl.sheet_names[0], header=0)
            except Exception:
                continue
            if df0.shape[1] >= 22 and "%" in str(df0.iloc[0, 3]):
                bonds_file = (path, xl)

    # Bonds
    bond_table_df = pd.DataFrame()
    if bonds_file:
        bonds_path, bonds_xl = bonds_file
        raw = pd.read_excel(bonds_path, sheet_name=bonds_xl.sheet_names[0], header=0)
        to_date = lambda s: pd.to_datetime(s, errors="coerce").dt.date
        bond_table_df = pd.DataFrame({
            "Issuer":         raw.iloc[:, 2],
            "Bond":           raw.iloc[:, 3].astype(str).str.replace(r"(\b\d{1,2})-(\d{4}\b)", r"\1/\2", regex=True).str.strip(),
            "Seniority":      raw.iloc[:, 10],
            "Moody's":        raw.iloc[:, 11],
            "S&P Rating":     "",
            "Issue Date":     to_date(raw.iloc[:, 5]),
            "Tenor":          pd.to_numeric(raw.iloc[:, 7], errors="coerce"),
            "Maturity":       to_date(raw.iloc[:, 6]),
            "UST":            raw.iloc[:, 8],
            "Coupon":         pd.to_numeric(raw.iloc[:, 4], errors="coerce"),
            "Amount Issued":  pd.to_numeric(raw.iloc[:, 9], errors="coerce"),
            "T-Spread":       pd.to_numeric(raw.iloc[:, 14], errors="coerce"),
        })
        bond_table_df["UST"] = bond_table_df["UST"].apply(norm_ust)

    # Adjusted (history)
    series_map = {}
    for h_path, h_xl in history_files:
        hist = pd.read_excel(h_path, sheet_name="Combined Level History", header=0)
        if hist.empty or hist.shape[1] < 2: continue
        date_col = pd.to_datetime(hist.iloc[:, 0], errors="coerce")
        t_cols = [c for c in hist.columns if isinstance(c, str) and re.search(r"\bT[- ]?Spread\b", c, flags=re.IGNORECASE)]
        for col in t_cols:
            base_label = re.sub(r"\s*T[- ]?Spread\s*$", "", col, flags=re.IGNORECASE).strip()
            base_label = re.sub(r"(\b\d{1,2})-(\d{4}\b)", r"\1/\2", base_label)
            parsed = parse_bond_label(base_label)
            canon = canonical_label(parsed) if parsed else base_label
            s = pd.to_numeric(hist[col], errors="coerce")
            s.index = date_col
            s = s[~s.index.isna()]
            s = s[~s.index.duplicated(keep="last")]
            series_map[canon] = series_map.get(canon, pd.Series(dtype=float)).combine_first(s) if canon in series_map else s

    if series_map:
        all_dates = sorted(set().union(*[s.index for s in series_map.values()]))
        adjusted_df = pd.DataFrame({"Date": pd.to_datetime(all_dates).date})
        for col_name in sorted(series_map.keys()):
            adjusted_df[col_name] = series_map[col_name].reindex(all_dates).values
    else:
        adjusted_df = pd.DataFrame(columns=["Date"])

    # TRACE (optional)
    trace_frames = []
    def _norm_party(x): return "" if pd.isna(x) else str(x).strip().upper()
    for h_path, h_xl in history_files:
        for sname in h_xl.sheet_names:
            if not sname.startswith("Trades "): continue
            try:
                df_tr = pd.read_excel(h_path, sheet_name=sname, header=0)
            except Exception:
                continue
            if df_tr.empty or df_tr.shape[1] < 14: continue
            date_ser      = pd.to_datetime(df_tr.iloc[:, 0], errors="coerce").dt.date
            qty_ser       = pd.to_numeric(df_tr.iloc[:, 1], errors="coerce")
            buysell_ser   = df_tr.iloc[:, 2].astype(str).str.strip()
            tspread_ser   = pd.to_numeric(df_tr.iloc[:, 3], errors="coerce")
            price_ser     = pd.to_numeric(df_tr.iloc[:, 5], errors="coerce")
            reporting_ser = df_tr.iloc[:, 8].apply(_norm_party)
            contra_ser    = df_tr.iloc[:, 9].apply(_norm_party)
            mask = reporting_ser.isin(["", "D"]) & contra_ser.isin(["", "C"])
            df_out = pd.DataFrame({
                "Date": date_ser, "Bond": "", "Quantity": qty_ser, "Buy/Sell": buysell_ser,
                "T-Spread": tspread_ser, "Price": price_ser,
            })
            df_out = df_out[mask & df_out["Date"].notna()]
            if not df_out.empty: trace_frames.append(df_out)
    trace_df = pd.concat(trace_frames, ignore_index=True) if trace_frames else pd.DataFrame(
        columns=["Date","Bond","Quantity","Buy/Sell","T-Spread","Price"]
    )

    return bond_table_df, adjusted_df, trace_df

# =========================
# CLEANED workbook
# =========================
def build_cleaned_workbook(bond_table_df: pd.DataFrame, adjusted_df: pd.DataFrame, trace_df: pd.DataFrame) -> str:
    global OUT_XLSX
    OUT_XLSX = os.path.join(OUTPUT_DIR, f"Cleaned {FS_DATE}.xlsx")
    with pd.ExcelWriter(OUT_XLSX, engine="openpyxl", datetime_format="yyyy-mm-dd") as writer:
        bond_table_df.to_excel(writer, sheet_name="Bond Table", index=False)
        adjusted_df.to_excel(writer, sheet_name="Adjusted", index=False)
        trace_df.to_excel(writer, sheet_name="TRACE", index=False)
    print(f"📘 Wrote: {OUT_XLSX}")
    return OUT_XLSX

# =========================
# Utilities from Bond Table
# =========================
def _maps_from_bond_table(bond_table_df: pd.DataFrame):
    canon_issue_map, canon_ust_map = {}, {}
    if isinstance(bond_table_df, pd.DataFrame) and not bond_table_df.empty:
        for _, r in bond_table_df[["Bond","Issue Date"]].dropna(subset=["Bond"]).iterrows():
            label = re.sub(r"(\b\d{1,2})-(\d{4}\b)", r"\1/\2", str(r["Bond"]).strip())
            parsed = parse_bond_label(label); canon = canonical_label(parsed) if parsed else None
            if not canon: continue
            iss_dt = pd.to_datetime(r["Issue Date"], errors="coerce")
            if pd.isna(iss_dt): continue
            prev = canon_issue_map.get(canon)
            if prev is None or iss_dt > prev: canon_issue_map[canon] = iss_dt
        for _, r in bond_table_df[["Bond","UST"]].dropna(subset=["Bond"]).iterrows():
            label = re.sub(r"(\b\d{1,2})-(\d{4}\b)", r"\1/\2", str(r["Bond"]).strip())
            parsed = parse_bond_label(label); canon = canonical_label(parsed) if parsed else None
            if not canon: continue
            canon_ust_map[canon] = norm_ust(r["UST"])
    return canon_issue_map, canon_ust_map

# =========================
# TARGET workbook (same charts logic as v4.3)
# =========================
def pick_month_window(dates: pd.Series, target_series: pd.Series, peer_avg_series: pd.Series) -> tuple[date, date]:
    dts = pd.to_datetime(dates, errors="coerce").dt.date
    df = pd.DataFrame({"Date": dts, "t": pd.to_numeric(target_series, errors="coerce"),
                       "pav": pd.to_numeric(peer_avg_series, errors="coerce")}).dropna(subset=["Date"])
    if df.empty:
        today = date.today()
        start = date(today.year, today.month, 1)
        end   = (date(today.year + (1 if today.month==12 else 0), 1 if today.month==12 else today.month+1, 1) - timedelta(days=1))
        return start, end
    df["y"] = [d.year for d in df["Date"]]; df["m"] = [d.month for d in df["Date"]]
    tgt_m = set(df.loc[df["t"].notna(), ["y","m"]].itertuples(index=False, name=None))
    pav_m = set(df.loc[df["pav"].notna(), ["y","m"]].itertuples(index=False, name=None))
    both = sorted(tgt_m & pav_m)
    if both:
        ys, ms = both[0]; start = date(ys, ms, 1)
    else:
        ft = sorted(tgt_m)[0] if tgt_m else None
        fp = sorted(pav_m)[0] if pav_m else None
        if ft and fp: yx, mx = max(ft, fp); start = date(yx, mx, 1)
        elif ft:      yx, mx = ft;          start = date(yx, mx, 1)
        elif fp:      yx, mx = fp;          start = date(yx, mx, 1)
        else:         start = min(df["Date"])
    either = sorted(tgt_m | pav_m)
    ye, me = either[-1]
    end = date(ye, me, 1)
    if me == 12: end = date(ye, 12, 31)
    else:        end = date(ye, me+1, 1) - timedelta(days=1)
    all_dates = set(df["Date"])
    if end not in all_dates:
        fwd = [d for d in all_dates if d >= end]; back = [d for d in all_dates if d <= end]
        if fwd: end = min(fwd)
        elif back: end = max(back)
    if start > end: start, end = min(all_dates), max(all_dates)
    return start, end

def _write_bond_table_target(writer, bond_table_df, target_by_ust):
    # (same as in v4.3; omitted here for brevity in explanation)
    cols = ["Issuer","Bond","Seniority","Moody's","S&P Rating","Issue Date","Tenor",
            "Maturity","UST","Coupon","Quantity","T-Spread"]
    df = bond_table_df.copy().rename(columns={"Amount Issued":"Quantity"})
    df = df[cols]
    df["Issue Date"] = pd.to_datetime(df["Issue Date"], errors="coerce").dt.date
    df["Maturity"]   = pd.to_datetime(df["Maturity"], errors="coerce").dt.date
    df["Tenor"]      = pd.to_numeric(df["Tenor"], errors="coerce")
    for c in ["Coupon","Quantity","T-Spread"]: df[c] = pd.to_numeric(df[c], errors="coerce")
    def bond_left(s):
        s = str(s) if pd.notna(s) else ""
        m = re.search(r"%", s);  return s[:m.end()] if m else s.strip()
    def fmt_m_yyyy(d):  return "" if pd.isna(d) or d is None else f"{d.month}/{d.year}"
    df["Bond"] = [f"{bond_left(b)} {fmt_m_yyyy(m)}".strip() for b, m in zip(df["Bond"], df["Maturity"])]
    df["UST"] = df["UST"].apply(norm_ust)
    def ust_key(u):
        s = str(u)
        if s == "Perp": return (10**9, s)
        m = re.match(r"(\d+)", s);  return (int(m.group(1)) if m else 10**8, s)
    df["__ustkey"] = df["UST"].map(ust_key)
    df.sort_values(by=["__ustkey","Tenor","Issuer"], ascending=[True, True, True], inplace=True)
    wb = writer.book
    ws = wb.add_worksheet("Bond Table")
    ws.hide_gridlines(2)
    blue = "#1F497D"
    prehdr = wb.add_format({'bg_color': blue})
    hdr = wb.add_format({'bold': True,'font_color': 'white','bg_color': blue,
                         'font_name': 'Calibri','font_size': 11,
                         'align': 'center','valign': 'vcenter','underline': 33})
    f_label = wb.add_format({'bold': True,'italic': True,'align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_left  = wb.add_format({'align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_date  = wb.add_format({'num_format':'m/d/yyyy','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_tenor = wb.add_format({'num_format':'0.0"y"','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_cpn   = wb.add_format({'num_format':'0.000','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_amt   = wb.add_format({'num_format':'#,##0.00" B"','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_tspr  = wb.add_format({'num_format':'0.0','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_tgt_left = wb.add_format({'bold': True,'bg_color':'#DCE6F1','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_tgt_date = wb.add_format({'bold': True,'bg_color':'#DCE6F1','num_format':'m/d/yyyy','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_tgt_ten  = wb.add_format({'bold': True,'bg_color':'#DCE6F1','num_format':'0.0"y"','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_tgt_cpn  = wb.add_format({'bold': True,'bg_color':'#DCE6F1','num_format':'0.000','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_tgt_amt  = wb.add_format({'bold': True,'bg_color':'#DCE6F1','num_format':'#,##0.00" B"','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_tgt_tspr = wb.add_format({'bold': True,'bg_color':'#DCE6F1','num_format':'0.0','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    widths = [30,28,10,8,10,12,9,12,8,8,12,9]
    start_row, start_col = 2, 2
    for j in range(len(cols)): ws.write_blank(start_row, start_col + j, None, prehdr)
    for j, c in enumerate(cols): ws.write(start_row + 1, start_col + j, c, hdr)
    for j, w in enumerate(widths): ws.set_column(start_col + j, start_col + j, w)
    target_by_ust_norm = {norm_ust(k): v for k, v in (target_by_ust or {}).items()}
    r = start_row + 2 + 1
    for ust, g in df.groupby("UST", sort=False):
        g = g.copy()
        g["__canon"] = None
        for i, rr in g.iterrows():
            parsed = parse_bond_label(str(rr["Bond"]))
            g.at[i,"__canon"] = canonical_label(parsed) if parsed else None
        target_canon = target_by_ust_norm.get(ust)
        if target_canon and target_canon in set(g["__canon"].dropna()):
            tgt_df = g[g["__canon"] == target_canon]
            peers  = g[g["__canon"] != target_canon].sort_values(["Tenor","Issuer"], ascending=[True, True])
            g = pd.concat([tgt_df, peers], ignore_index=True)
        else:
            g = g.sort_values(["Tenor","Issuer"], ascending=[True, True]).reset_index(drop=True)
        ws.set_row(r, 3.0); r += 1
        ws.write(r, start_col + 0, ust, f_label); r += 1
        ws.set_row(r, 3.0); r += 1
        first_row = r; last_row = r
        for _, rr in g.iterrows():
            is_tgt = (rr["__canon"] == target_canon)
            L, D, Tn, Cp, Am, Ts = (
                (f_tgt_left, f_tgt_date, f_tgt_ten, f_tgt_cpn, f_tgt_amt, f_tgt_tspr)
                if is_tgt else
                (f_left, f_date, f_tenor, f_cpn, f_amt, f_tspr)
            )
            c = start_col
            ws.write(r, c+0, rr.get("Issuer",""), L)
            ws.write(r, c+1, rr.get("Bond",""),   L)
            ws.write(r, c+2, rr.get("Seniority",""), L)
            ws.write(r, c+3, rr.get("Moody's",""),   L)
            ws.write(r, c+4, rr.get("S&P Rating",""),L)
            idt, mdt = rr.get("Issue Date"), rr.get("Maturity")
            if pd.notna(idt): ws.write_datetime(r, c+5, datetime.combine(idt, dtime.min), D)
            else:             ws.write(r, c+5, "", D)
            ten = rr.get("Tenor")
            if pd.notna(ten): ws.write_number(r, c+6, float(ten), Tn)
            else:             ws.write(r, c+6, "", Tn)
            if pd.notna(mdt): ws.write_datetime(r, c+7, datetime.combine(mdt, dtime.min), D)
            else:             ws.write(r, c+7, "", D)
            ws.write(r, c+8, rr.get("UST",""), L)
            coup = rr.get("Coupon")
            if pd.notna(coup): ws.write_number(r, c+9, float(coup), Cp)
            else:              ws.write(r, c+9, "", Cp)
            amt = rr.get("Quantity")
            if pd.notna(amt): ws.write_number(r, c+10, float(amt)/1_000_000_000.0, f_amt)
            else:             ws.write(r, c+10, "", f_amt)
            ts = rr.get("T-Spread")
            if pd.notna(ts):  ws.write_number(r, c+11, float(ts), f_tspr)
            else:             ws.write(r, c+11, "", f_tspr)
            last_row = r
            r += 1
        stripe_fmt = wb.add_format({'bg_color':'#F2F2F2'})
        ws.conditional_format(first_row, start_col, last_row, start_col + len(cols) - 1, {
            'type': 'formula',
            'criteria': f'=MOD(ROW()-{first_row},2)=0',
            'format': stripe_fmt
        })

def build_target_workbook(bond_table_df: pd.DataFrame, adjusted_df: pd.DataFrame) -> str | None:
    global TARGET_XLSX
    if adjusted_df.empty or adjusted_df.shape[1] < 2:
        print("⚠️  No Adjusted data found; cannot build Target workbook.")
        return None
    ticker = input("Enter the target TICKER (e.g., JPM): ").strip().upper()
    if not ticker:
        print("⚠️  No ticker provided; skipping Target.")
        return None
    canon_issue_map, canon_ust_map = _maps_from_bond_table(bond_table_df)
    by_bucket = {}
    for canon in adjusted_df.columns[1:]:
        m = re.match(r"^\s*([A-Z0-9\.\-]+)", str(canon))
        if not m: continue
        if m.group(1) != ticker: continue
        u = canon_ust_map.get(canon)
        if u: by_bucket.setdefault(norm_ust(u), []).append(canon)
    if not by_bucket:
        print(f"⚠️  No bonds with UST buckets found for '{ticker}'.")
        return None
    def _pick_latest_issue(canon_list):
        if not canon_list: return None
        with_dates = [(c, canon_issue_map.get(c)) for c in canon_list]
        dated = [t for t in with_dates if t[1] is not None]
        if dated: return max(dated, key=lambda t: (t[1], t[0]))[0]
        def _cp(c):
            m = re.match(r"^\s*([A-Z0-9\.\-]+)\s+(\d+(?:\.\d+)?)%", c)
            return float(m.group(2)) if m else -1.0
        return max(canon_list, key=lambda c: (_cp(c), c))
    target_by_ust = {}
    for u, cands in by_bucket.items():
        tcanon = _pick_latest_issue(cands)
        if tcanon: target_by_ust[u] = tcanon
    TARGET_XLSX = os.path.join(OUTPUT_DIR, f"Target {FS_DATE}.xlsx")
    with pd.ExcelWriter(TARGET_XLSX, engine="xlsxwriter", datetime_format="yyyy-mm-dd") as tw:
        _write_bond_table_target(tw, bond_table_df, target_by_ust)
        def m_order(txt):
            s = norm_ust(txt)
            if s == "Perp": return 10**9
            m = re.match(r"(\d+)", s);  return int(m.group(1)) if m else 10**8
        for ust, cands in sorted(by_bucket.items(), key=lambda kv: m_order(kv[0])):
            tcanon = target_by_ust.get(ust)
            if not tcanon: continue
            peers_all = []
            for c in adjusted_df.columns[1:]:
                if c == tcanon: continue
                if norm_ust(_maps_from_bond_table(bond_table_df)[1].get(c)) == ust:
                    peers_all.append(c)
            peers_by_ticker = defaultdict(list)
            for c in peers_all:
                m = re.match(r"^\s*([A-Z0-9\.\-]+)", str(c))
                if m: peers_by_ticker[m.group(1)].append(c)
            peer_list = []
            for _, lst in peers_by_ticker.items():
                keep = _pick_latest_issue(lst)
                if keep and keep != tcanon: peer_list.append(keep)
            out = pd.DataFrame({"Date": adjusted_df["Date"]})
            out[tcanon] = pd.to_numeric(adjusted_df.get(tcanon), errors="coerce")
            peer_cols = []
            for i, c in enumerate(sorted(peer_list), start=1):
                name = f"Peer Bond #{i} — {c}"
                out[name] = pd.to_numeric(adjusted_df.get(c), errors="coerce")
                peer_cols.append(name)
            if peer_cols:
                cnt = out[peer_cols].notna().sum(axis=1)
                out["Peer Avg"] = out[peer_cols].mean(axis=1)
                out.loc[cnt < 2, "Peer Avg"] = np.nan
            else:
                out["Peer Avg"] = np.nan
            disp_target_col = f"Target Bond — {tcanon}"
            out = out.rename(columns={tcanon: disp_target_col})
            start_d, end_d = pick_month_window(out["Date"], out[disp_target_col], out["Peer Avg"])
            mask = (out["Date"] >= start_d) & (out["Date"] <= end_d)
            out_win = out.loc[mask].reset_index(drop=True)
            sheet_name = ust
            out_win.to_excel(tw, sheet_name=sheet_name, index=False, startrow=29, startcol=0)
            wb, ws = tw.book, tw.sheets[sheet_name]
            nrows = len(out_win)
            tgt_idx = out_win.columns.get_loc(disp_target_col)
            pav_idx = out_win.columns.get_loc("Peer Avg")
            peer_idx = [out_win.columns.get_loc(c) for c in peer_cols]
            def add_series(chart, col_idx, name, is_target=False, width=2.0):
                chart.add_series({
                    'name': name,
                    'categories': [sheet_name, 30, 0, 30 + nrows - 1, 0],
                    'values':     [sheet_name, 30, col_idx, 30 + nrows - 1, col_idx],
                    'line': {'color': 'red' if is_target else None, 'width': width}
                })
            y_pg_min, y_pg_max = y_range_5s_push10(
                np.concatenate([pd.to_numeric(out_win[disp_target_col], errors="coerce").values] +
                               [pd.to_numeric(out_win[c], errors="coerce").values for c in peer_cols])
            )
            y_pa_min, y_pa_max = y_range_5s_push10(
                np.concatenate([pd.to_numeric(out_win[disp_target_col], errors="coerce").values,
                                pd.to_numeric(out_win["Peer Avg"], errors="coerce").values])
            )
            ch_pg = base_chart(wb)
            configure_date_axis_monthly(ch_pg, start_d, end_d)
            ch_pg.set_y_axis({'min': y_pg_min, 'max': y_pg_max, 'label_position': 'low',
                              'major_unit': 10, 'major_gridlines': {'visible': False},
                              'minor_gridlines': {'visible': False}})
            for idx in peer_idx:
                pretty = out_win.columns[idx].split('—', 1)[-1].strip() if '—' in out_win.columns[idx] else out_win.columns[idx]
                add_series(ch_pg, idx, pretty, False, 2.0)
            add_series(ch_pg, tgt_idx, disp_target_col.replace("Target Bond — ", ""), True, 3.0)
            ws.insert_chart(1, 0, ch_pg, {'x_scale': 1.1, 'y_scale': 1.0})
            ch_pa = base_chart(wb)
            configure_date_axis_monthly(ch_pa, start_d, end_d)
            ch_pa.set_y_axis({'min': y_pa_min, 'max': y_pa_max, 'label_position': 'low',
                              'major_unit': 10, 'major_gridlines': {'visible': False},
                              'minor_gridlines': {'visible': False}})
            add_series(ch_pa, pav_idx, "Peer Avg", False, 2.0)
            add_series(ch_pa, tgt_idx, disp_target_col.replace("Target Bond — ", ""), True, 2.0)
            ws.insert_chart(12, 0, ch_pa, {'x_scale': 1.1, 'y_scale': 1.0})
    print(f"📘 Wrote: {TARGET_XLSX}")
    return TARGET_XLSX

# =========================
# EVENT ANALYSIS workbook — FIXED date windows per bond
# =========================
def _write_event_bond_table(writer, bond_table_df: pd.DataFrame, selected_canons: list[str]):
    """Flat Bond Table for Event Analysis; bold UST column; even-row striping."""
    cols = ["Issuer","Bond","Seniority","Moody's","S&P Rating","Issue Date","Tenor",
            "Maturity","UST","Coupon","Quantity","T-Spread"]
    df = bond_table_df.copy().rename(columns={"Amount Issued":"Quantity"})
    # filter to selected
    def to_canon(s):
        parsed = parse_bond_label(str(s));  return canonical_label(parsed) if parsed else None
    df["__canon"] = df["Bond"].apply(to_canon)
    df = df[df["__canon"].isin(set(selected_canons))].copy()
    # types
    df["Issue Date"] = pd.to_datetime(df["Issue Date"], errors="coerce").dt.date
    df["Maturity"]   = pd.to_datetime(df["Maturity"], errors="coerce").dt.date
    df["Tenor"]      = pd.to_numeric(df["Tenor"], errors="coerce")
    for c in ["Coupon","Quantity","T-Spread"]: df[c] = pd.to_numeric(df[c], errors="coerce")
    # recompute Bond display & UST
    def bond_left(s):
        s = str(s) if pd.notna(s) else ""
        m = re.search(r"%", s);  return s[:m.end()] if m else s.strip()
    def fmt_m_yyyy(d):  return "" if pd.isna(d) or d is None else f"{d.month}/{d.year}"
    df["Bond"] = [f"{bond_left(b)} {fmt_m_yyyy(m)}".strip() for b, m in zip(df["Bond"], df["Maturity"])]
    df["UST"] = df["UST"].apply(norm_ust)
    # sort
    df.sort_values(by=["Tenor","Issuer"], ascending=[True, True], inplace=True)
    df = df[cols]
    # write
    wb = writer.book;  ws = wb.add_worksheet("Bond Table");  ws.hide_gridlines(2)
    blue = "#1F497D"
    prehdr = wb.add_format({'bg_color': blue})
    hdr = wb.add_format({'bold': True,'font_color': 'white','bg_color': blue,
                         'font_name': 'Calibri','font_size': 11,'align':'center','valign':'vcenter','underline':33})
    f_left   = wb.add_format({'align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_left_b = wb.add_format({'align':'left','valign':'vcenter','font_name':'Calibri','font_size':11,'bold':True})
    f_date   = wb.add_format({'num_format':'m/d/yyyy','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_tenor  = wb.add_format({'num_format':'0.0"y"','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_cpn    = wb.add_format({'num_format':'0.000','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_amt    = wb.add_format({'num_format':'#,##0.00" B"','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    f_tspr   = wb.add_format({'num_format':'0.0','align':'left','valign':'vcenter','font_name':'Calibri','font_size':11})
    widths = [30,28,10,8,10,12,9,12,8,8,12,9]
    start_row, start_col = 2, 2
    for j in range(len(cols)): ws.write_blank(start_row, start_col + j, None, prehdr)
    for j, c in enumerate(cols): ws.write(start_row + 1, start_col + j, c, hdr)
    for j, w in enumerate(widths): ws.set_column(start_col + j, start_col + j, w)
    r = start_row + 2 + 1
    ws.set_row(r, 3.0);  r += 1  # one pad row
    first_data_row = r;  last_data_row = r-1
    for _, rr in df.iterrows():
        c = start_col
        ws.write(r, c+0, rr.get("Issuer",""), f_left)
        ws.write(r, c+1, rr.get("Bond",""),   f_left)
        ws.write(r, c+2, rr.get("Seniority",""), f_left)
        ws.write(r, c+3, rr.get("Moody's",""),   f_left)
        ws.write(r, c+4, rr.get("S&P Rating",""),f_left)
        idt, mdt = rr.get("Issue Date"), rr.get("Maturity")
        if pd.notna(idt): ws.write_datetime(r, c+5, datetime.combine(idt, dtime.min), f_date)
        else:             ws.write(r, c+5, "", f_date)
        ten = rr.get("Tenor")
        if pd.notna(ten): ws.write_number(r, c+6, float(ten), f_tenor)
        else:             ws.write(r, c+6, "", f_tenor)
        if pd.notna(mdt): ws.write_datetime(r, c+7, datetime.combine(mdt, dtime.min), f_date)
        else:             ws.write(r, c+7, "", f_date)
        ws.write(r, c+8, rr.get("UST",""), f_left_b)  # bold UST
        coup = rr.get("Coupon")
        if pd.notna(coup): ws.write_number(r, c+9, float(coup), f_cpn)
        else:              ws.write(r, c+9, "", f_cpn)
        amt = rr.get("Quantity")
        if pd.notna(amt): ws.write_number(r, c+10, float(amt)/1_000_000_000.0, f_amt)
        else:             ws.write(r, c+10, "", f_amt)
        ts = rr.get("T-Spread")
        if pd.notna(ts):  ws.write_number(r, c+11, float(ts), f_tspr)
        else:             ws.write(r, c+11, "", f_tspr)
        last_data_row = r;  r += 1
    stripe_fmt = wb.add_format({'bg_color':'#F2F2F2'})
    ws.conditional_format(first_data_row, start_col, last_data_row, start_col + len(cols) - 1, {
        'type': 'formula',
        'criteria': f'=MOD(ROW()-{first_data_row},2)=0',
        'format': stripe_fmt
    })

def build_event_analysis_workbook(bond_table_df: pd.DataFrame, adjusted_df: pd.DataFrame) -> str | None:
    global EVENT_XLSX
    if adjusted_df.empty or adjusted_df.shape[1] < 2:
        print("⚠️  No Adjusted data; cannot build Event Analysis.")
        return None

    ticker = input("Enter the issuer TICKER for Event Analysis (e.g., JPM): ").strip().upper()
    if not ticker:
        print("⚠️  No ticker provided; skipping Event Analysis.")
        return None

    event_date_str = input("Enter the EVENT DATE (e.g., 6/15/2025): ").strip()
    try:
        event_date = pd.to_datetime(event_date_str, errors="raise").date()
    except Exception:
        print("❗ Could not parse that date. Use m/d/yyyy like 6/15/2025.")
        return None

    # maps
    canon_issue_map, canon_ust_map = _maps_from_bond_table(bond_table_df)

    # discover bonds of ticker; select latest per UST
    by_bucket = {}
    for canon in adjusted_df.columns[1:]:
        m = re.match(r"^\s*([A-Z0-9\.\-]+)", str(canon))
        if not m: continue
        if m.group(1) != ticker: continue
        u = canon_ust_map.get(canon)
        if u: by_bucket.setdefault(norm_ust(u), []).append(canon)

    if not by_bucket:
        print(f"⚠️  No bonds for ticker '{ticker}' found in Adjusted.")
        return None

    def _pick_latest_issue(canon_list):
        if not canon_list: return None
        with_dates = [(c, canon_issue_map.get(c)) for c in canon_list]
        dated = [t for t in with_dates if t[1] is not None]
        if dated: return max(dated, key=lambda t: (t[1], t[0]))[0]
        # fallback: highest coupon
        def _cp(c):
            m = re.match(r"^\s*([A-Z0-9\.\-]+)\s+(\d+(?:\.\d+)?)%", c)
            return float(m.group(2)) if m else -1.0
        return max(canon_list, key=lambda c: (_cp(c), c))

    selected = []
    for u, cands in by_bucket.items():
        tcanon = _pick_latest_issue(cands)
        if tcanon: selected.append((u, tcanon))

    if not selected:
        print("⚠️  No bonds selected for Event Analysis.")
        return None

    EVENT_XLSX = os.path.join(OUTPUT_DIR, f"Event Analysis {FS_DATE}.xlsx")
    with pd.ExcelWriter(EVENT_XLSX, engine="xlsxwriter", datetime_format="yyyy-mm-dd") as tw:
        wb = tw.book

        # 0) Flat Bond Table
        _write_event_bond_table(tw, bond_table_df, [canon for _, canon in selected])

        # 1..n) One sheet per bond — UST-named tabs (unique)
        used_names = set()
        for idx, (ust, canon) in enumerate(sorted(selected, key=lambda t: (999999 if t[0]=="Perp" else int(t[0].replace("y",""))))):
            base_name = norm_ust(ust) if ust else "Bond"
            sheet_name = base_name
            k = 2
            while sheet_name in used_names:
                sheet_name = f"{base_name}-{k}"; k += 1
            used_names.add(sheet_name)

            # Build per-bond frame from Adjusted
            dates = pd.to_datetime(adjusted_df["Date"], errors="coerce").dt.date
            vals  = pd.to_numeric(adjusted_df.get(canon), errors="coerce")
            df = pd.DataFrame({"Date": dates, "T-Spread": vals})
            df["Prior"] = df["T-Spread"]
            df.loc[df["Date"] >= event_date, "Prior"] = np.nan
            df["Post"]  = df["T-Spread"]
            df.loc[df["Date"] < event_date, "Post"] = np.nan

            # ---- Compute bond-specific first/last non-NaN dates ----
            valid_mask = df["T-Spread"].notna().values
            if np.any(valid_mask):
                inds = np.flatnonzero(valid_mask)
                bond_first_date = df["Date"].iloc[inds[0]]
                bond_last_date  = df["Date"].iloc[inds[-1]]
            else:
                bond_first_date = event_date
                bond_last_date  = event_date

            # ---- Write table (charts at top, table starts row 30) ----
            start_row = 29
            df.to_excel(tw, sheet_name=sheet_name, index=False, startrow=start_row, startcol=0)
            ws = tw.sheets[sheet_name]
            # put canonical label + bold UST at top
            ws.write(0, 0, canon)
            bold_fmt = wb.add_format({'bold': True})
            ws.write(0, 1, base_name, bold_fmt)

            nrows = len(df)
            y_min, y_max = y_range_5s_push10(df["T-Spread"].values)
            color = EA_COLORS[idx % len(EA_COLORS)]

            # ---- Chart 1: All data split by event (bond-specific x range) ----
            ch1 = base_chart(wb)
            configure_date_axis_monthly(ch1, bond_first_date, bond_last_date)
            ch1.set_y_axis({'min': y_min, 'max': y_max, 'label_position': 'low',
                            'major_unit': 10, 'major_gridlines': {'visible': False},
                            'minor_gridlines': {'visible': False}})
            # Prior: solid line
            ch1.add_series({
              'name': f"Prior-{canon}",
              'categories': [sheet_name, start_row+1, 0, start_row+nrows, 0],
              'values':     [sheet_name, start_row+1, 2, start_row+nrows, 2],
              'line': {'dash_type': 'solid', 'width': 2.0, 'color': color},
              'marker': {'type': 'none'}
            })

            # Post: square-dot dashed line (same color as Prior), NOT markers-only
            ch1.add_series({
              'name': f"Post-{canon}",
              'categories': [sheet_name, start_row+1, 0, start_row+nrows, 0],
              'values':     [sheet_name, start_row+1, 3, start_row+nrows, 3],
              'line': {'dash_type': 'square_dot', 'width': 2.0, 'color': color},
              'marker': {'type': 'none'}
            })

            ws.insert_chart(1, 0, ch1, {'x_scale': 1.1, 'y_scale': 1.0})

            # ---- Chart 2: Post-only (bond-specific x range) ----
            ch2 = base_chart(wb)
            # first post date for this bond = first non-NaN on/after event_date
            post_mask = (df["Date"] >= event_date) & df["T-Spread"].notna()
            if post_mask.any():
                post_first_date = df.loc[post_mask, "Date"].iloc[0]
                post_last_date  = df.loc[df["T-Spread"].notna(), "Date"].iloc[-1]
            else:
                post_first_date = event_date
                post_last_date  = bond_last_date
            configure_date_axis_monthly(ch2, post_first_date, post_last_date)
            ch2.set_y_axis({'min': y_min, 'max': y_max, 'label_position': 'low',
                            'major_unit': 10, 'major_gridlines': {'visible': False},
                            'minor_gridlines': {'visible': False}})
            # locate first row index >= event_date (even if NaNs, we start from nearest)
            first_post_idx = df.index[df["Date"] >= post_first_date]
            if len(first_post_idx) == 0:
                post_start_row = start_row+1
            else:
                post_start_row = start_row + 1 + int(first_post_idx[0])
            post_end_row = start_row + nrows
            ch2.add_series({
                'name': canon,
                'categories': [sheet_name, post_start_row, 0, post_end_row, 0],
                'values':     [sheet_name, post_start_row, 1, post_end_row, 1],
                'line': {'width': 2.0, 'color': color},
                'marker': {'type': 'none'}
            })
            ws.insert_chart(12, 0, ch2, {'x_scale': 1.1, 'y_scale': 1.0})

    print(f"📘 Wrote: {EVENT_XLSX}")
    return EVENT_XLSX

# =========================
# Runner
# =========================
if __name__ == "__main__":
    raw_dir = get_raw_folder()
    print(f"📂 Scanning: {raw_dir}")
    bond_table_df, adjusted_df, trace_df = ingest_bonds_and_history(raw_dir)

    # Build Cleaned
    build_cleaned_workbook(bond_table_df, adjusted_df, trace_df)

    # Build Target?
    if input("\nBuild TARGET workbook? (y/n): ").strip().lower() in {"y","yes"}:
        build_target_workbook(bond_table_df, adjusted_df)

    # Build Event Analysis?
    if input("\nBuild EVENT ANALYSIS workbook? (y/n): ").strip().lower() in {"y","yes"}:
        build_event_analysis_workbook(bond_table_df, adjusted_df)

    print(f"\n📂 Output directory: {os.path.abspath(OUTPUT_DIR)}")

    # ---- Colab auto-downloads (safe, prints what was queued) ----
    try:
        from google.colab import files as colab_files  # type: ignore
        print("\n⬇️  Prompting download dialogs...")
        downloaded = []
        for varname in ["OUT_XLSX", "TARGET_XLSX", "EVENT_XLSX"]:
            try:
                if varname in globals() and globals()[varname]:
                    colab_files.download(globals()[varname])
                    downloaded.append(globals()[varname])
            except Exception:
                pass
        if downloaded:
            print("\n📘 Successfully queued for download:")
            for f in downloaded:
                print("  -", f)
        else:
            print("\n⚠️ No workbooks available to download.")
    except Exception:
        # Not in Colab; just print paths
        print("\n(Info) Not in Colab; skipping auto-downloads.")
        if OUT_XLSX:    print("  Cleaned:", OUT_XLSX)
        if TARGET_XLSX: print("  Target: ", TARGET_XLSX)
        if EVENT_XLSX:  print("  Event:  ", EVENT_XLSX)

    print("\n🎯 Done (v4.3.1).")
